In [1]:
class lagrangianParticles:
    def __init__(self, numberParticles):
        
        ## current position and velocity
        self.X = np.empty( (numberParticles, 3) )
        self.U = np.empty( (numberParticles, 3) )
        self.Uflip = np.empty( (numberParticles, 3) )
        self.Upic = np.empty( (numberParticles, 3) )
        
        ## mass, density and volume
        self.M = np.empty( (numberParticles, 3) )
        
        ## deformation gradients and placeholders for num. evolution
        self.Fe = np.zeros( (numberParticles, 3, 3) )
        self.Fp = np.zeros( (numberParticles, 3, 3) ) 
        self.F = np.zeros( (numberParticles, 3, 3) ) ## placeholder when doing update
        self.Feh = np.zeros( (numberParticles, 3, 3) ) ## placeholder when doing update
        self.Fph = np.zeros( (numberParticles, 3, 3) ) ## placeholder when doing update 
        self.Fu = np.zeros( (numberParticles, 3, 3) )  ## placeholder when doing update
    
        self.lameParam = np.zeros( (numberParticles, 2) ) ## both μ and λ
        self.svdu = np.zeros( (numberParticles, 3, 3) )
        self.svds = np.zeros( (numberParticles, 3 ) )
        self.svdv = np.zeros( (numberParticles, 3, 3) )
        self.Ψ = np.zeros( (numberParticles, 1) )
        self.σ = np.zeros( (numberParticles, 3, 3) )
        
        self.ijk = np.zeros( (numberParticles, 3), dtype=np.int64 )

        
# THE FOLLOWING LINES CAN BE ADJUSTED TO OBTAIN DIFFERENT INITIAL 
# CONDITIONS FOR THE SNOW, I.E., FOR A SPHERE YOU COULD USE A WHILE
# LOOP THAT CHECKS THAT THE RANDOMLY STARTED POSITIONS ARE INSIDE THE
# SPHERE, OR COULD CREATE A FUNCTION TO INITIALIZE THE DIFFERENT SHAPES
# SUCH AS BLOCKS OR SPHERE
        ## initialize lagrangian particle values
        for x in self.X:
            x[:] = np.random.randn(3) % .25
        self.X[:] = (self.X - .125)
        self.X[:,1] = self.X[:,1] + .5 - .2

        for u in self.U:
            u[:] = 0.
            u[1] = -g*sqrt(.2/2/g)

        for m in self.M:
            m[0] = max(.015/4 + np.random.randn(1)/200000, 0)  ## get mass
            m[1] = 0.
            m[2] = 0.

        ## initialize deformation gradient, which initially is undeformed, so we use identity
        for f in self.Fe:
            f[:] = np.eye(3) # + np.random.randn((3))/1e7
        for f in self.Fp:
            f[:] = np.eye(3)
            
            
            
            
            
    def saveTxt(self, case, i):
        fname = case + "X" + str(i)
        np.save(fname, self.X)
        
        fname = case + "U" + str(i)
        np.save(fname, self.U)
        
        fname = case + "M" + str(i)
        np.save(fname, self.M)
        
    def sort3D(self, mx, my, mz):
        partitions = []
        iix = np.argsort(self.X[:,0])
        iix = np.array_split(iix, mx)
        for ix in iix:
            iiy = np.argsort(self.X[ix,1])
            iiy = np.array_split(iiy, my)
            for iy in iiy:
                iiz = np.argsort(self.X[ix[iy],2])
                iiz = np.array_split(iiz, mz)
                for iz in iiz:
                    partitions.append(ix[iy[iz]])
        return partitions
    
    def loadData(self, xpath, upath, mpath):
        self.X[:] = np.load(xpath)
        self.U[:] = np.load(upath)
        self.M[:] = np.load(mpath)
            
